In [ ]:
import bmll2 as b2
b2.get_file('modules/auxiliary_functions.py')

In [ ]:
import auxiliary_functions as af

import random
import math
import pandas as pd
import numpy as np
from pandas import StringDtype

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import LogFormatterSciNotation

## Plotting of the aggregated impact curve of the 100 stocks

In [ ]:
%%time
b2.get_file('test_data/homo_4_immediate(100).csv')
impact_data = pd.read_csv('homo_4_immediate(100).csv', parse_dates = ['Date', 'Start time', 'End time'])
impact_data['Start time']    = pd.to_datetime(impact_data['Start time'], format = 'mixed')
impact_data['End time']      = pd.to_datetime(impact_data['End time'], format = 'mixed')
impact_data['duration(min)'] = ((impact_data['End time'] - impact_data['Start time']).dt.total_seconds() / 60).round(6)
# MTN has a weird glitch where we have negative values for volume traded. All these have a trade sign of 0 so need to be removed
# should probably do this in the processing stage but isnt a big deal here
impact_data = impact_data[impact_data['trade sign'] != 0]

# takes about 26 seconds to load (homo_4_immediate)
# takes about 2 minutes to load

In [ ]:
# sanity check. There should not be any data where the volume traded is negative
impact_data[impact_data['volume traded'] < 0]


In [ ]:
n_intervals = 40

x_meta = impact_data['volume traded'] / impact_data['20 AD volume']
y_meta = impact_data['impact(simple)'] / impact_data['20 AD volatility']

bins_meta           = np.logspace(np.log10(x_meta.min()), np.log10(x_meta.max()), n_intervals, base = 10, endpoint = True)
x_bins_meta         = pd.cut(x_meta, bins = bins_meta) 

y_bins_vals_meta    = y_meta.groupby(x_bins_meta, observed = False).mean()
x_bins_centers_meta = [interval.mid for interval in y_bins_vals_meta.index.categories]

alpha          = 0.5
sqrt_law       = alpha * np.sqrt(x_bins_centers_meta)


In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))

ax.set_xscale('log')
ax.set_yscale('log')

ax.plot(x_bins_centers_meta, y_bins_vals_meta, marker = 'o', color = 'blue', label = 'Metaorder impact curve')
ax.plot(x_bins_centers_meta, sqrt_law, linestyle = '--', color = 'red', label = r'0.5 $\times \sqrt{Q/V_{D}}$')

ax.set_xlabel(r'Q/$V_{D}$', fontsize = 12)
ax.set_ylabel(r'I(Q/$V_{D}$)/$\sigma$', fontsize = 12)

# force scientific notation 10^n ticks
ax.xaxis.set_major_formatter(LogFormatterSciNotation(labelOnlyBase = False))
ax.yaxis.set_major_formatter(LogFormatterSciNotation(labelOnlyBase = False))

plt.legend()

plt.savefig('top_100_impact_curve(20ADV).pdf')
b2.put_file('top_100_impact_curve(20ADV).pdf', 'figures')
b2.get_file('figures/top_100_impact_curve(20ADV).pdf')

plt.show()

## Plotting of the impact curves for GRT and GFI

In [ ]:
N                   = 4
trader_distribution = 'homogenous'
identifier          = f'{trader_distribution}_{N}'

In [ ]:
GRT = impact_data[impact_data['RIC'] == 'GRT']
GFI = impact_data[impact_data['RIC'] == 'GFI']

In [ ]:
n_intervals = 40
bins        = np.logspace(-6, 0, n_intervals, base = 10, endpoint = True)

In [ ]:
x_GRT = GRT['volume traded'] / GRT['20 AD volume']
y_GRT = GRT['impact(simple)'] / GRT['20 AD volatility']

#bins_GRT           = np.logspace(np.log10(x_GRT.min()), np.log10(x_GRT.max()), n_intervals, base = 10, endpoint = True)
x_bins_GRT         = pd.cut(x_GRT, bins = bins) 

y_bins_vals_GRT    = y_GRT.groupby(x_bins_GRT, observed = False).mean()
x_bins_centers_GRT = [interval.mid for interval in y_bins_vals_GRT.index.categories]

In [ ]:
x_GFI = GFI['volume traded'] / GFI['20 AD volume']
y_GFI = GFI['impact(simple)'] / GFI['20 AD volatility']

#bins_GFI           = np.logspace(np.log10(x_GFI.min()), np.log10(x_GFI.max()), n_intervals, base = 10, endpoint = True)
x_bins_GFI         = pd.cut(x_GFI, bins = bins) 

y_bins_vals_GFI    = y_GFI.groupby(x_bins_GFI, observed = False).mean()
x_bins_centers_GFI = [interval.mid for interval in y_bins_vals_GFI.index.categories]

In [ ]:
alpha       = 0.5
sqrt_law    = alpha * np.sqrt(x_bins_centers_GRT)

fig, ax = plt.subplots(figsize = (6, 6))

#ax.set_title('Recovering the square root law', fontsize = 14)
ax.set_xscale('log')
ax.set_yscale('log')

ax.plot(x_bins_centers_GRT, y_bins_vals_GRT, marker = 'o', color = 'olive', label = 'GRT impact curve')
ax.plot(x_bins_centers_GFI, y_bins_vals_GFI, marker = 'o', color = 'purple', label = 'GFI impact curve')
ax.plot(x_bins_centers_GRT, sqrt_law, linestyle = '--', color = 'red', label = r'0.5 $\times \sqrt{Q/V_{D}}$')

ax.set_xlabel(r'Q/$V_{D}$', fontsize = 12)
ax.set_ylabel(r'I(Q/$V_{D}$)/$\sigma$', fontsize = 12)

# force scientific notation 10^n ticks
ax.xaxis.set_major_formatter(LogFormatterSciNotation(labelOnlyBase = False))
ax.yaxis.set_major_formatter(LogFormatterSciNotation(labelOnlyBase = False))

plt.legend()

plt.savefig(f'GRT_GFI_impact_curve_{identifier}.pdf')
b2.put_file(f'GRT_GFI_impact_curve_{identifier}.pdf', 'figures')
b2.get_file(f'figures/GRT_GFI_impact_curve_{identifier}.pdf')

plt.show()